In [0]:
%run ../includes/common_functions

In [0]:
%run ../includes/configuration

In [0]:
from pyspark.sql.functions import sum, count, countDistinct, desc, asc, rank, when, col
from pyspark.sql.window import Window

In [0]:
results_df = spark.read.parquet("/mnt/formuladlgen/presentation/race_results")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2296443588599013> in <module> 
 ----> 1 results_df = spark . read . parquet ( "/mnt/formuladlgen/presentation/race_results" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: dbfs:/mnt/formuladlgen/presentation/race_results

In [0]:
grouped_df = results_df\
        .groupBy("race_year", "team")\
        .agg(sum("points").alias("total_points"), countDistinct("race_name").alias("total_races"))

In [0]:
constructorRankSpec = Window.partitionBy("race_year").orderBy(desc("total_points"))
grouped_df.withColumn("rank", rank().over(constructorRankSpec)).show(100)

In [0]:
constructor_standings_df = results_df\
.groupBy("race_year", "team")\
.agg(sum("points").alias("total_points"), count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(constructor_standings_df)

In [0]:
constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins")) 
final_constructor_rankings_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
display(final_constructor_rankings_df)

In [0]:
final_constructor_rankings_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_rankings")

In [0]:
dbutils.notebook.exit("Success")